#Partie 1: Rappel du problème

Nous devons mettre sur pied un modèle de comptage du nombre de safou sur une image.Et pour pouvoir compter , il faut tout d'abord détecter les safous , le nombre total de safou détecté sur une image correspondra au nombre de safou total présent sur l'image.
Nous allons mettre sur pied un modèle de détection des safous.
Après arecherches , analyse et test , nous n'avons pas trouvé de modèle directement utilisable pour les raison suivantes:
Les modèles essayés actuels ont des très mauvais résultats et 95% n'ont pas été entrainer sur les safous .

Pour se faire nous sommes passés par les étapes suivantes

1) Acquisition du dataset et analyse
Nous avons eu un dataset de près de 400 images.
Après anaylse approfondie , nous avons constaté que la taille d'un safou est très petit par rapport à la dimension de l'image(moins de 2%). Nous sommes donc dans la détection des objets petit :les small object détection. Nous avons donc choisi d'utilisé Yolov11 + Sahi


2) Labellisation
A l'aide  de l'outil label studio , nous avons labelisé nos safous.
Vous avez pu voir la complexité d'une labelisation , et le temps que cela pourrait prendre.
Savez aujourdhui que avant de commencer à labeliser une image , il faut s'arrurer que son nom respecte les normes : pas d'espace , ni de caractères accentués ,symboles... sinon des erreurs de tout bord peuvent survenir lors de l'exportation des données labelisées.


3) Quels formats d'exportation choisir ?

le choix du format d'exportation dépend du modèle qui va recevoir notre jeux de donnée. Etant donné que nous avons choisi YoloV11 ,D'après sa documentation ce dernier admet plusieurs formats dont le format du dataset coco8.
Il s'agira donc pour nous de mettre sur pied un dataset final qui respectera la structure du dataset coco8 https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco8.yaml


4) Structure du dataset coco8

Il se compose de deux dossiers coco8.yaml et le dossier coco8

  a) le fichier coco8.yaml est un fichier de configuration qui contient

  - path : le chemin d'accès vers le dossier coco8
  - train : le chemin d'accès relatif vers le dossier train
  -val :--//-- de validation
  -test : --//-- de test , très utile pour l'initialisation des hyperparamètres

  -names:
    id:label1 exemple : 0:person , correspond à l'id du label person
  -dowload : le lien vers le dossier coco8 si il est en ligne.

  nb : coco8 n'est que le nom du dataset , vous pouvez donc le remplacer par tout autre nom en respectant les dépendances .


  b) le dossier coco8
  ce dossier contient deux sous dossiers : images et labels.
  - images: contient les images déja partitionnées en sous dossiers( train val test(optionnel))
      -train : contient les images qui seront utilisées lors de l'entrainement.
      -val : --//-- de l'évaluation du modèle
      -test : --//-- pour l'initialisation des hyper-paramètres.

  nb :Toujours vous assurer d'avoir les bonnes extensions des images selon les modèles . Dans notre cas .jpg

  -labels : contient les fichiers .txt de labelisation de chaque image
      -train : fichiers de labelisations correspondants aux images du dossier images/train
      il en est de même pour val ,et test.

  Attention : le nom du fichier de labelisation doit correspondre au nom exact de l'image correspondante


  c)  apperçu d'un ficier de labelisation coco8/labels/train/nom_image.txt

    - 45 0.479492 0.688771 0.955609 0.5955
    - 45 0.736516 0.247188 0.498875 0.476417
    - 50 0.637063 0.732938 0.494125 0.510583
    - 45 0.339438 0.418896 0.678875 0.7815
    - 49 0.646836 0.132552 0.118047 0.0969375
    - 49 0.773148 0.129802 0.0907344 0.0972292
    - 49 0.668297 0.226906 0.131281 0.146896
    - 49 0.642859 0.0792187 0.148063 0.148062

  Chaque colone est délimitée par un espace

  Dans la première colone , nous avons l'id des labels
  les colonnes 2,3,4,5 suivantes correspondent aux positions des points de références de chaque bounding box(rectangle, carré ,.... de labelistion)





5) Exportation des données de label studio

Nous utilisons le format "YOLO with image" de label studio pour exporter car c'est ce dernier qui se rapproche le plus du fichier de labelisation précédent.
Il est exporté sous forme de .zip



C'EST ICI QUE NOTRE TRAVAIL COMMENCE

Executez les cellules suivantes au fur et à mesure , ne touchez pas au code sauf si vous juger cela indispensable.





#Partie 2 : structuration du dataset exporter selon le dataset coco8

## Aperçu du dataset initial exporté

ce dernier de compose de deux fichiers et 02 dossiers

- classes.txt comportant le nom des classes et id : il sera supprimé car nous n'avons qu'une seule classe : safou
- notes.json : comporte les notes et données écrites lors de la labelisaition : nous n'en n'aurons pas besoin . il sera supprimé
- labels : comporte les labels des images en .txt selon la même structuration vu précédemmment
- images : comporte les images correspondantes.

##Chargement du dataset exporté à partir de google drive

En supposant que vous avez déja uploader le dataset dézippé  dans google drive , nous allons le rendre accessible dans cet environement via le code suivant .
nb : renommer votre dossier en safou1 par exemple selon le numéro qui vous a été attribué

In [ ]:
import os


# 1. Dossier de base sur ta machine
# Mets ici le dossier dans lequel tu veux chercher tes datasets
# Par exemple : r"C:\Users\Franck\Documents" sous Windows
# ou "/home/franck/Documents" sous Linux
base_dir = r"E:/ProjetVerification&ValidationDocuments/dataset"

# Nom du dataset que vous recherchez
nom_dataset =input("cni")

# Construction du chemin complet.
# Notez que le point de montage dans Colab est généralement '/content/gdrive'
# et le dossier principal de votre Drive est 'MyDrive'.
chemin_complet = os.path.join(base_dir, nom_dataset)

# 2. Vérification de l'existence du chemin
if os.path.exists(chemin_complet):
    print("Le dataset a été trouvé à :", chemin_complet)
    # Vous pouvez ajouter ici le code pour charger ou utiliser votre dataset
else:
    print("Dataset introuvable. Veuillez vérifier les points suivants :")
    print("1. Assurez-vous que le Drive est monté (drive.mount('/content/gdrive') a fonctionné).")
    print("2. Vérifiez que le nom : '" + nom_dataset + "' correspond exactement (majuscules/minuscules incluses) au fichier ou dossier dans 'Mon Drive'.")
    print("3. Si vous avez un espace dans 'My Drive' dans votre chemin local, Colab le gère généralement comme '/content/gdrive/MyDrive', mais vérifiez la structure des fichiers à gauche.")
    print("Chemin recherché:", chemin_complet)

KeyboardInterrupt: Interrupted by user

##suppression de fichiers inutile : classes.txt et notes.json

In [32]:
import os
from google.colab import drive



# Définition du chemin complet du dossier (basé sur la structure Colab)
dossier_dataset = chemin_complet
# Liste des fichiers à supprimer DANS ce dossier
fichiers_a_supprimer = ["classes.txt", "notes.json"]


# ----------------------------------------------------
# 2. Logique de suppression
# ----------------------------------------------------

print(f"Tentative de suppression des fichiers dans le dossier : {dossier_dataset}")

# Vérifier si le dossier du dataset existe avant de continuer
if not os.path.isdir(dossier_dataset):
    print(f"❌ Erreur : Le dossier du dataset '{dossier_dataset}' est introuvable. Veuillez vérifier le nom.")
else:
    for fichier in fichiers_a_supprimer:
        # Construction du chemin complet pour chaque fichier
        chemin_complet_fichier = os.path.join(dossier_dataset, fichier)

        # Vérification et suppression sécurisée
        if os.path.exists(chemin_complet_fichier):
            try:
                os.remove(chemin_complet_fichier)
                print(f"✅ Fichier '{fichier}' supprimé avec succès.")
            except Exception as e:
                # Gérer les erreurs de permission ou autres problèmes
                print(f"❌ Erreur lors de la suppression de '{fichier}' : {e}")
        else:
            print(f"⚠️ Fichier '{fichier}' non trouvé dans le dossier. (Rien à supprimer)")

Tentative de suppression des fichiers dans le dossier : /content/gdrive/MyDrive/safou2
✅ Fichier 'classes.txt' supprimé avec succès.
✅ Fichier 'notes.json' supprimé avec succès.


## Creation du dossier racine et migration des dossiers labels et images

In [33]:
import os
import sys

def creer_dossier(chemin_dossier: str):
    """
    Tente de créer le dossier spécifié. Si le dossier existe, ne fait rien.
    Si la création échoue (ex: problèmes de permission), arrête l'exécution du script.

    Args:
        chemin_dossier (str): Le chemin complet du dossier à créer/vérifier.
    """
    print(f"Tentative de vérification/création du dossier : {chemin_dossier}")
    try:
        # os.makedirs crée tous les dossiers intermédiaires nécessaires
        # exist_ok=True : n'émet pas d'erreur si le dossier existe déjà
        os.makedirs(chemin_dossier, exist_ok=True)
        print(f"✅ Dossier créé/vérifié avec succès : {chemin_dossier}")
    except Exception as e:
        print(f"❌ Erreur critique lors de la création du dossier : {e}")
        print("L'exécution du script est interrompue.")
        # Arrête le programme
        sys.exit(1)


In [34]:
import os
import shutil
from google.colab import drive




NOM_DATASET = chemin_complet

NOM_NOUVEAU_DOSSIER = os.path.basename(chemin_complet)

# Chemin de base de votre dataset (où se trouvent 'labels' et 'images')
DATASET_PATH = os.path.join("/content/gdrive/MyDrive", NOM_DATASET)


# ----------------------------------------------------
# 2. Définition des chemins
# ----------------------------------------------------

# Chemin du nouveau dossier à créer
NOUVEAU_DOSSIER_PATH = os.path.join(DATASET_PATH, NOM_NOUVEAU_DOSSIER)

# Chemins des dossiers sources
SOURCE_LABELS_PATH = os.path.join(DATASET_PATH, "labels")
SOURCE_IMAGES_PATH = os.path.join(DATASET_PATH, "images")

# ----------------------------------------------------
# 3. Opérations : Création et Déplacement
# ----------------------------------------------------

print(f"Dossier parent cible : {DATASET_PATH}")


creer_dossier(NOUVEAU_DOSSIER_PATH)
# Liste des dossiers à déplacer
dossiers_a_deplacer = [
    (SOURCE_LABELS_PATH, "labels"),
    (SOURCE_IMAGES_PATH, "images")
]

# 2. Couper-Coller (Déplacer) les dossiers 'labels' et 'images'
for source_path, nom_dossier in dossiers_a_deplacer:

    # Définir la destination complète (le dossier 'labels' ou 'images' ira dans 'nouveau_sous_dossier')
    destination_path = os.path.join(NOUVEAU_DOSSIER_PATH, nom_dossier)

    if os.path.isdir(source_path):
        try:
            # shutil.move effectue l'opération de "couper-coller" (déplacement)
            shutil.move(source_path, destination_path)
            print(f"✅ Dossier '{nom_dossier}' déplacé avec succès vers : {NOUVEAU_DOSSIER_PATH}")
        except Exception as e:
            print(f"❌ Erreur lors du déplacement de '{nom_dossier}' : {e}")
    else:
        print(f"⚠️ Dossier source '{nom_dossier}' non trouvé à : {source_path}")

print("\nProcessus terminé.")

Dossier parent cible : /content/gdrive/MyDrive/safou2
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2
✅ Dossier 'labels' déplacé avec succès vers : /content/gdrive/MyDrive/safou2/safou2
✅ Dossier 'images' déplacé avec succès vers : /content/gdrive/MyDrive/safou2/safou2

Processus terminé.


## remplacement des tirets par les underscores dans les noms des fichiers




In [35]:



def renommer_fichiers_tiret_underscore(repertoire_cible: str):
    """
    Parcourt un répertoire et renomme tous les fichiers en remplaçant
    les tirets (-) par des underscores (_).

    Args:
        repertoire_cible (str): Le chemin du répertoire à traiter.
    """
    if not os.path.isdir(repertoire_cible):
        print(f"❌ ERREUR : Le chemin spécifié est introuvable ou n'est pas un dossier : {repertoire_cible}")
        return

    compteur_renommes = 0
    print(f"Démarrage du renommage dans le dossier : {repertoire_cible}")

    # os.listdir() récupère les noms de tous les éléments (fichiers et dossiers)
    for nom_ancien in os.listdir(repertoire_cible):

        # Construire le chemin complet de l'élément (pour vérifier si c'est un fichier)
        chemin_complet_ancien = os.path.join(repertoire_cible, nom_ancien)

        # 1. Vérifier si c'est un fichier et si son nom contient un tiret
        if os.path.isfile(chemin_complet_ancien) and '-' in nom_ancien:

            # 2. Créer le nouveau nom en remplaçant les tirets par des underscores
            nom_nouveau = nom_ancien.replace('-', '_')

            # Construire le chemin complet du nouveau fichier
            chemin_complet_nouveau = os.path.join(repertoire_cible, nom_nouveau)

            # 3. Effectuer l'opération de renommage
            try:
                os.rename(chemin_complet_ancien, chemin_complet_nouveau)
                compteur_renommes += 1
                # Afficher le changement (facultatif)
                print(f"  RENOMMÉ : {nom_ancien} -> {nom_nouveau}")

            except Exception as e:
                print(f"❌ ERREUR lors du renommage de {nom_ancien} : {e}")

        elif os.path.isdir(chemin_complet_ancien):
            # Ignorer les sous-dossiers
            pass

        # Ignorer les fichiers qui n'ont pas de tiret

    print("\n--- Bilan du renommage ---")
    if compteur_renommes > 0:
        print(f"🎉 Succès ! Total de {compteur_renommes} fichiers renommés.")
    else:
        print("ℹ️ Aucun fichier contenant un tiret (-) n'a été trouvé et renommé.")






In [36]:
import os
SOURCE_LABELS_PATH = os.path.join(DATASET_PATH,os.path.basename(DATASET_PATH), "labels")
SOURCE_IMAGES_PATH = os.path.join(DATASET_PATH,os.path.basename(DATASET_PATH), "images")

renommer_fichiers_tiret_underscore(SOURCE_LABELS_PATH)
renommer_fichiers_tiret_underscore(SOURCE_IMAGES_PATH)

Démarrage du renommage dans le dossier : /content/gdrive/MyDrive/safou2/safou2/labels
  RENOMMÉ : f32e9484-A3PXL_20250511_063147305.txt -> f32e9484_A3PXL_20250511_063147305.txt
  RENOMMÉ : dd316262-A1PXL_20250511_061618363.txt -> dd316262_A1PXL_20250511_061618363.txt

--- Bilan du renommage ---
🎉 Succès ! Total de 2 fichiers renommés.
Démarrage du renommage dans le dossier : /content/gdrive/MyDrive/safou2/safou2/images
  RENOMMÉ : f32e9484-A3PXL_20250511_063147305.jpg -> f32e9484_A3PXL_20250511_063147305.jpg
  RENOMMÉ : dd316262-A1PXL_20250511_061618363.jpg -> dd316262_A1PXL_20250511_061618363.jpg

--- Bilan du renommage ---
🎉 Succès ! Total de 2 fichiers renommés.


## Uniformisation de l'id de labelisation dans les fichiers de labelisation

Nous définissons l'id de labelisation du safou , l'id 1. ainsi , tous les éléments de la première colonne des fichiers de labelisation seront remplacés par 1.

In [37]:
import os
from typing import List

# ----------------------------------------------------------------------
# FONCTION UTILITAIRE
# ----------------------------------------------------------------------

def _modifier_fichier_label(
    chemin_fichier_label: str,
    nouvelle_classe_id: str,
    separateur: str = ' '
) -> bool:
    """
    Modifie le premier élément (indice de classe) de chaque ligne d'un fichier
    texte. Retourne True si la modification a réussi, False sinon.
    """

    if not os.path.exists(chemin_fichier_label):
        print(f"  [Erreur] Fichier non trouvé : {chemin_fichier_label}")
        return False

    lignes_modifiees: List[str] = []
    modification_effectuee = False

    # 1. Lecture du fichier
    try:
        with open(chemin_fichier_label, 'r') as f:
            lignes = f.readlines()
    except Exception as e:
        print(f"  [Erreur] Lecture impossible : {e}")
        return False

    # 2. Traitement ligne par ligne
    for ligne in lignes:
        ligne_strip = ligne.strip()

        if not ligne_strip:
            lignes_modifiees.append('\n')
            continue

        elements = ligne_strip.split(separateur)

        if len(elements) >= 1 and elements[0].isdigit(): # Vérifie si c'est un indice de classe
            # Remplacer le premier élément
            if elements[0] != nouvelle_classe_id:
                elements[0] = nouvelle_classe_id
                modification_effectuee = True

            # Reconstruire la ligne et conserver le saut de ligne original
            nouvelle_ligne = separateur.join(elements) + '\n'
            lignes_modifiees.append(nouvelle_ligne)
        else:
            # Conserver la ligne originale si elle est mal formatée ou vide
            lignes_modifiees.append(ligne)

    # 3. Écriture du contenu modifié (si une modification a eu lieu)
    if modification_effectuee:
        try:
            with open(chemin_fichier_label, 'w') as f:
                f.writelines(lignes_modifiees)
            return True
        except Exception as e:
            print(f"  [Erreur] Écriture impossible : {e}")
            return False

    return True # Retourne True si tout s'est bien passé même sans modification


# ----------------------------------------------------------------------
# FONCTION PRINCIPALE : Itération sur le répertoire
# ----------------------------------------------------------------------

def modifier_classe_pour_repertoire_labels(
    repertoire_labels: str,
    nouvelle_classe_id: str,
    extension_cible: str = '.txt'
):
    """
    Itère sur tous les fichiers d'un répertoire et modifie leur première colonne.

    Args:
        repertoire_labels (str): Le chemin du dossier contenant les fichiers de labels.
        nouvelle_classe_id (str): Le nouvel identifiant de classe (doit être une chaîne, ex: '1').
        extension_cible (str): L'extension des fichiers à modifier (par défaut: '.txt').
    """
    if not os.path.isdir(repertoire_labels):
        print(f"❌ ERREUR : Le répertoire est introuvable : {repertoire_labels}")
        return

    compteur_traites = 0
    compteur_succes = 0

    print(f"Démarrage de la modification des labels dans : {repertoire_labels}")
    print(f"Nouvel ID de classe cible : {nouvelle_classe_id}")

    # Parcourir tous les éléments dans le répertoire
    for nom_fichier in os.listdir(repertoire_labels):

        # Filtrer par extension et s'assurer que ce n'est pas un dossier
        if nom_fichier.endswith(extension_cible) and os.path.isfile(os.path.join(repertoire_labels, nom_fichier)):

            chemin_complet = os.path.join(repertoire_labels, nom_fichier)
            compteur_traites += 1

            # Appeler la fonction utilitaire pour modifier le fichier
            if _modifier_fichier_label(chemin_complet, nouvelle_classe_id):
                compteur_succes += 1

    print("\n--- Bilan de l'opération ---")
    print(f"Total de fichiers {extension_cible} trouvés : {compteur_traites}")
    print(f"Total de fichiers traités avec succès : {compteur_succes}")




In [38]:
----------------------------------------------

# Assurez-vous d'avoir ce répertoire avec des fichiers .txt de labels
DOSSIER_DE_LABELS = SOURCE_LABELS_PATH
NOUVEL_ID_CLASSE = "1" # Mettre la nouvelle valeur de classe (ex: '0', '1', '15'...)

# Exemple d'appel :
modifier_classe_pour_repertoire_labels(DOSSIER_DE_LABELS, NOUVEL_ID_CLASSE)

Démarrage de la modification des labels dans : /content/gdrive/MyDrive/safou2/safou2/labels
Nouvel ID de classe cible : 1

--- Bilan de l'opération ---
Total de fichiers .txt trouvés : 2
Total de fichiers traités avec succès : 2


## Creation du fichier de configuration .yaml

In [2]:
!pip install pyyaml

In [39]:
import os
# La librairie yaml n'est pas strictement nécessaire ici si on écrit juste une chaîne,
# mais on la garde au cas où vous changeriez d'avis.
# Assurez-vous d'avoir fait !pip install pyyaml avant d'exécuter.
import yaml


# 3. Récupération du nom du dataset (le dernier élément du chemin)
NOM_NOUVEAU_DOSSIER = os.path.basename(chemin_complet)

# ----------------------------------------------------
# 1. Configuration des chemins (CORRIGÉ)
# ----------------------------------------------------

# Le répertoire cible est le chemin de base du dataset
REPERTOIRE_CIBLE = chemin_complet

# Le nom du fichier à créer (ex: safou1.yaml)
NOM_FICHIER = NOM_NOUVEAU_DOSSIER + '.yaml'

# Construction du chemin complet du fichier (DOIT venir après REPERTOIRE_CIBLE et NOM_FICHIER)
fichierYaml = os.path.join(REPERTOIRE_CIBLE, NOM_FICHIER)

# -------------------------------------------------------------------------
# --- 2. Définition du contenu YAML (Chaîne formatée) ---
# -------------------------------------------------------------------------

# La f-string injecte la valeur de la variable NOM_NOUVEAU_DOSSIER
contenu_yaml_string = f"""
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: {NOM_NOUVEAU_DOSSIER}  # dataset root dir (le dossier safou1 ou autre)
train: images/train # train images (relative to 'path')
val: images/val # val images (relative to 'path')
test: images/test # test images (optional)

# Classes
nc: 1 # nombre de classes
names:
  1: safou

# Download script/URL (optional)
#download: lien du dataset distant
"""

# ----------------------------------------------------
# 3. Création et écriture du fichier
# ----------------------------------------------------

# 1. S'assurer que le répertoire cible existe
try:
    # REPERTOIRE_CIBLE est ici "/content/gdrive/MyDrive/safou1"
    os.makedirs(REPERTOIRE_CIBLE, exist_ok=True)
    print(f"✅ Répertoire cible vérifié : {REPERTOIRE_CIBLE}")
except Exception as e:
    print(f"❌ Erreur lors de la vérification/création du répertoire : {e}")
    # Arrêter l'exécution si le dossier n'existe pas
    exit()

# 2. Écrire la chaîne dans le fichier YAML
try:
    # On utilise 'chemin_complet_fichier' pour le chemin du fichier
    with open(fichierYaml, 'w') as file:
        # On écrit la chaîne directement, sans passer par yaml.dump (qui nécessite un dict)
        file.write(contenu_yaml_string.strip())

    print(f"🎉 Fichier '{NOM_FICHIER}' créé avec succès à : {fichierYaml}")
    print("\n--- Contenu écrit dans le fichier ---")
    print(contenu_yaml_string)

except Exception as e:
    print(f"❌ Erreur lors de l'écriture du fichier YAML : {e}")

✅ Répertoire cible vérifié : /content/gdrive/MyDrive/safou2
🎉 Fichier 'safou2.yaml' créé avec succès à : /content/gdrive/MyDrive/safou2/safou2.yaml

--- Contenu écrit dans le fichier ---

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: safou2  # dataset root dir (le dossier safou1 ou autre)
train: images/train # train images (relative to 'path')
val: images/val # val images (relative to 'path')
test: images/test # test images (optional)

# Classes
nc: 1 # nombre de classes
names:
  0: safou

# Download script/URL (optional)
#download: lien du dataset distant



Allez vérifier les modifications au fur et à mesure dans votre google drive

## division du dataset en 03 parties train ,val, test

In [40]:
def recuperer_noms_base_fichiers(dossier_a_scanner: str) -> list:
    """
    Récupère le nom de base (sans extension) de tous les fichiers
    uniques trouvés dans un dossier.

    Args:
        dossier_a_scanner (str): Le chemin du dossier à scanner.

    Returns:
        list: Une liste de chaînes de caractères contenant les noms de base uniques.
    """
    if not os.path.isdir(dossier_a_scanner):
        print(f"❌ ERREUR : Le chemin spécifié est introuvable ou n'est pas un dossier : {dossier_a_scanner}")
        return []

    # Utiliser un set pour stocker les noms de base uniques (élimine les doublons)
    noms_base_uniques = set()

    for nom_complet in os.listdir(dossier_a_scanner):
        chemin_complet = os.path.join(dossier_a_scanner, nom_complet)

        # Vérifier que c'est bien un fichier et non un sous-dossier
        if os.path.isfile(chemin_complet):
            # os.path.splitext sépare le nom de base de l'extension
            nom_base, _ = os.path.splitext(nom_complet)
            noms_base_uniques.add(nom_base)

    # Convertir le set en liste avant de le retourner
    return list(noms_base_uniques)



In [41]:
import os
import random


# ----------------------------------------------------
# 1. Configuration (À ADAPTER !)
# ----------------------------------------------------

# IMPORTANT : REMPLACEZ CE CHEMIN par le chemin de votre dossier 'images' ou 'labels'
DOSSIER_A_SCANNER = os.path.join(chemin_complet,os.path.basename(chemin_complet),'labels')

# Ratios de division (Test: 10%, Validation: 30%, Entraînement: 60%)
RATIO_TEST = 0.10
RATIO_VAL = 0.30


# ----------------------------------------------------
# 2. Récupérer, mélanger et diviser
# ----------------------------------------------------

# Récupérer la liste des noms de base (SANS extension)
noms_base_fichiers = recuperer_noms_base_fichiers(DOSSIER_A_SCANNER)

if not noms_base_fichiers:
    print("Liste des fichiers est vide. Arrêt.")
    exit()

# ⚠️ ÉTAPE CRUCIALE : Mélanger la liste pour garantir une répartition aléatoire
random.shuffle(noms_base_fichiers)

nombre_total_fichiers = len(noms_base_fichiers)
print(f"Nombre total de noms de base de fichiers uniques trouvés : {nombre_total_fichiers}")


# 3. Calculer les indices de division
idx_fin_test = int(nombre_total_fichiers * RATIO_TEST)
idx_fin_val = idx_fin_test + int(nombre_total_fichiers * RATIO_VAL)


# 4. Diviser la liste
liste_test = noms_base_fichiers[:idx_fin_test]
liste_val = noms_base_fichiers[idx_fin_test:idx_fin_val]
liste_train = noms_base_fichiers[idx_fin_val:]


# ----------------------------------------------------
# 5. Afficher les résultats
# ----------------------------------------------------

print("\n--- Répartition des noms de base de fichiers ---")
print(f"Liste TRAIN (60%) : {len(liste_train)} noms")
print(f"Liste VAL (30%) : {len(liste_val)} noms")
print(f"Liste TEST (10%) : {len(liste_test)} noms")

# Vérification finale
total_divise = len(liste_train) + len(liste_val) + len(liste_test)
print(f"\nVérification : Total divisé ({total_divise}) doit être égal au Total trouvé ({nombre_total_fichiers})")

Nombre total de noms de base de fichiers uniques trouvés : 2

--- Répartition des noms de base de fichiers ---
Liste TRAIN (60%) : 2 noms
Liste VAL (30%) : 0 noms
Liste TEST (10%) : 0 noms

Vérification : Total divisé (2) doit être égal au Total trouvé (2)


creation dossiers train dev et test dans les dossiers labels et images

In [42]:

dossier_label=os.path.join(chemin_complet,nom_dataset,"labels")
dossier_image=os.path.join(chemin_complet,nom_dataset,"images")

creer_dossier(os.path.join(dossier_label,"train"))
creer_dossier(os.path.join(dossier_label,"val"))
creer_dossier(os.path.join(dossier_label,"test"))


creer_dossier(os.path.join(dossier_image,"train"))
creer_dossier(os.path.join(dossier_image,"val"))
creer_dossier(os.path.join(dossier_image,"test"))



Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2/labels/train
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2/labels/train
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2/labels/val
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2/labels/val
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2/labels/test
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2/labels/test
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2/images/train
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2/images/train
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/safou2/safou2/images/val
✅ Dossier créé/vérifié avec succès : /content/gdrive/MyDrive/safou2/safou2/images/val
Tentative de vérification/création du dossier : /content/gdrive/MyDrive/saf

Répartition des fichiers dans les dossiers train val test

In [43]:
import os
import shutil
from typing import List
def deplacer_fichiers(
    liste_noms_base: List[str],
    repertoire_source: str,
    repertoire_destination: str,
    extensions_possibles: List[str] = ['.jpg', '.jpeg', '.png', '.txt', '.xml']
):
    """
    Déplace un ensemble de fichiers d'un répertoire à un autre.

    La fonction vérifie l'existence des fichiers en ignorant leur extension
    (en testant toutes les extensions fournies dans extensions_possibles).

    Args:
        liste_noms_base (List[str]): Liste des noms de fichiers SANS extension
                                     (ex: ['image1', 'image2']).
        repertoire_source (str): Chemin du répertoire où se trouvent les fichiers.
        repertoire_destination (str): Chemin du répertoire de destination.
        extensions_possibles (List[str]): Liste des extensions à tester.
    """
    print(f"Tentative de déplacement de {len(liste_noms_base)} fichiers...")

    # Assurer que le répertoire de destination existe
    try:
        os.makedirs(repertoire_destination, exist_ok=True)
    except Exception as e:
        print(f"❌ ERREUR : Impossible de créer le répertoire de destination '{repertoire_destination}'. Arrêt.")
        print(f"Détail : {e}")
        return # Arrêter la fonction

    fichiers_deplaces = 0
    fichiers_non_trouves = []
    for nom_base in liste_noms_base:
        fichier_trouve = False

        # 1. Tenter de trouver le fichier avec toutes les extensions possibles
        for ext in extensions_possibles:
            nom_complet_source = nom_base + ext
            chemin_source_complet = os.path.join(repertoire_source, nom_complet_source)

            if os.path.exists(chemin_source_complet):
                # Fichier trouvé !

                # 2. Définir le chemin de destination
                chemin_destination_complet = os.path.join(repertoire_destination, nom_complet_source)

                # 3. Déplacer (Couper-coller)
                try:
                    shutil.move(chemin_source_complet, chemin_destination_complet)
                    # print(f"  -> Déplacé : {nom_complet_source}") # Décommenter pour plus de détails
                    fichiers_deplaces += 1
                    fichier_trouve = True
                    break # Sortir de la boucle d'extensions, le fichier a été déplacé
                except Exception as e:
                    print(f"❌ ERREUR lors du déplacement de {nom_complet_source} : {e}")
                    fichier_trouve = True # Considérer trouvé mais erreur de move
                    break

        if not fichier_trouve:
            fichiers_non_trouves.append(nom_base)

    print("\n--- Bilan du déplacement ---")
    print(f"✅ Total déplacé : {fichiers_deplaces} fichiers.")

    if fichiers_non_trouves:
        print(f"⚠️ {len(fichiers_non_trouves)} fichiers de base non trouvés (avec aucune des extensions testées) :")
        # Afficher la liste si elle est petite, sinon la taille
        if len(fichiers_non_trouves) < 10:
            print(fichiers_non_trouves)
        else:
            print("Liste trop longue pour être affichée.")
    else:
        print("🎉 Tous les fichiers de la liste ont été trouvés et déplacés (ou déjà absents).")


In [44]:
#variables d'accès

dossier_image_train = os.path.join(dossier_image, "train")
dossier_image_val = os.path.join(dossier_image, "val")
dossier_image_test = os.path.join(dossier_image, "test")

dossier_label_train = os.path.join(dossier_label, "train")
dossier_label_val = os.path.join(dossier_label, "val")
dossier_label_test = os.path.join(dossier_label, "test")


print("--- Démarrage de la fonction de déplacement ---")
print("\nDébut répartition des fichiers images...")
deplacer_fichiers(liste_train,dossier_image,dossier_image_train,extensions_possibles=['.jpg', '.png', '.jpeg'])
deplacer_fichiers(liste_val,dossier_image,dossier_image_val,extensions_possibles=['.jpg', '.png', '.jpeg'])
deplacer_fichiers(liste_test,dossier_image,dossier_image_test,extensions_possibles=['.jpg', '.png', '.jpeg'])
print("\nFin de la répartition des fichiers images.")

print("\nDébut répartition des fichiers label...")
deplacer_fichiers(liste_train,dossier_label,dossier_label_train,extensions_possibles=['.txt'])
deplacer_fichiers(liste_val,dossier_label,dossier_label_val,extensions_possibles=['.txt'])
deplacer_fichiers(liste_test,dossier_label,dossier_label_test,extensions_possibles=['.txt'])


print("\nFin de la répartition des fichiers label..")

--- Démarrage de la fonction de déplacement ---

Début répartition des fichiers images...
Tentative de déplacement de 2 fichiers...

--- Bilan du déplacement ---
✅ Total déplacé : 2 fichiers.
🎉 Tous les fichiers de la liste ont été trouvés et déplacés (ou déjà absents).
Tentative de déplacement de 0 fichiers...

--- Bilan du déplacement ---
✅ Total déplacé : 0 fichiers.
🎉 Tous les fichiers de la liste ont été trouvés et déplacés (ou déjà absents).
Tentative de déplacement de 0 fichiers...

--- Bilan du déplacement ---
✅ Total déplacé : 0 fichiers.
🎉 Tous les fichiers de la liste ont été trouvés et déplacés (ou déjà absents).

Fin de la répartition des fichiers images.

Début répartition des fichiers label...
Tentative de déplacement de 2 fichiers...

--- Bilan du déplacement ---
✅ Total déplacé : 2 fichiers.
🎉 Tous les fichiers de la liste ont été trouvés et déplacés (ou déjà absents).
Tentative de déplacement de 0 fichiers...

--- Bilan du déplacement ---
✅ Total déplacé : 0 fichiers.

# A propos de l'auteur

Ing Mowoum Foupouagnigni Génie Informatique Promotion 2025 Ecole Nationale Supérieure Polytechnique de Yaoundé
695187698
foupouagnignimowoum@gmail.com

N'hésitez pas à me contacter et à me faire des retours de ce notebook ,aussi bien sur la forme , que le fond ...